In [1]:
from repath.utils.paths import project_root

slide_path = project_root() / 'data' / 'icaird_animal' / 'raw' / 'train' / 'slides' /'0.isyntax'

ModuleNotFoundError: No module named 'repath'

In [13]:
import repath.data.slides.libraries.pixelengine as pixelengine
import repath.data.slides.libraries.softwarerendercontext as softwarerendercontext
import repath.data.slides.libraries.softwarerenderbackend as softwarerenderbackend

render_context = softwarerendercontext.SoftwareRenderContext()
render_backend = softwarerenderbackend.SoftwareRenderBackend()
pe = pixelengine.PixelEngine(render_backend, render_context)

In [14]:
level = 5
image_index = 0

input = pe['in']
input.open(str(slide_path))
source_view = input.SourceView()
envelopes = source_view.dataEnvelopes(level)
rect = envelopes.asRectangles()[image_index]
rect

[64, 8640, 55904, 64960]

In [15]:
from repath.utils.geometry import Point

start_x, end_x, start_y, end_y = rect
origin = Point(start_x, start_y)
origin

Point(x=64, y=55904)

In [16]:
from typing import List

from repath.utils.geometry import Size
from repath.data.slides import Region

def region_to_range(region: Region) -> List:
    level, loc, size = region
    range = [loc.x + origin.x, loc.x + size.width,
             loc.y + origin.y, loc.y + size.height, level]
    return range

    level: int
    location: Point
    size: Size

range = region_to_range(Region(5, Point(1000, 1000), Size(256, 256)))
range

[1064, 1256, 56904, 1256, 5]

In [ ]:
from concurrent import futures
from multiprocessing import cpu_count

in_regions = [Region(5, Point(1000, 1000), Size(256, 256)),
              Region(5, Point(3000, 3000), Size(256, 256))]

def get_region_size(reg):
    x_start, x_end, y_start, y_end, lvl = reg.range
    ranges = view.dimensionRanges(lvl)
    # this calculation ensures that the width and height are on
    # the pixel bounds for this level
    w = int(1 + (x_end - x_start) / ranges[0][1])
    h = int(1 + (y_end - y_start) / ranges[1][1])
    return w, h

def process_image(pixels, region_id, width, height):
    print(f"Processing region {region_id}")

ranges = [region_to_range(r) for r in regions]
pe_regions = source_view.requestRegions(in_regions, envelopes,
                                        True, [0,0,0],
                                        pe.BufferType.RGB)
jobs = []
with futures.ThreadPoolExecutor(max_workers=cpu_count()) as executor:
    while len(pe_regions) > 0:
        regions_ready = pe.waitAny(pe_regions)
        for region in regions_ready:
            print("Processing region: {region}")
            region_idx = ranges.index(region.range)  # where the region is in the list
            width, height = get_region_size(region)
            buffer_size = width * height * samples_per_pixel
            pixels = get_pixels(region, buffer_size)
            pe_regions.remove(region)
            jobs.append(executor.submit())

futures.wait(jobs, return_when=futures.ALL_COMPLETED)

images = read_regions(regions)

In [ ]:
def read_regions(self, regions: List[Region]) -> List[Image]:

    ranges = region_to_range()

    pass